## Imports

In [1]:
from JSU_lib import *

## Reading input data

In [2]:
LIGYSIS_sites = pd.read_pickle("./results/LIGYSIS_sites_DEF_TRANS.pkl")                                          # 2024
VNEGNN_pockets = pd.read_pickle("./results/VNEGNN_pockets_DEF_TRANS.pkl")                                        # 2024
IFSP_pockets = pd.read_pickle("./results/IFSP_pockets_DEF_TRANS.pkl")                                            # 2024
GRASP_pockets = pd.read_pickle("./results/GRASP_pockets_DEF_TRANS_RIGHT_CLUSTERING.pkl")                         # 2024

PURESNET_PRANK_CONS_pockets = pd.read_pickle("./results/puresnet_pranked_CONS_pockets_DEF_TRANS.pkl")            # 2021 + 2019
PURESNET_PRANK_pockets = pd.read_pickle("./results/puresnet_pranked_pockets_DEF_TRANS.pkl")                      # 2021 + 2015
PURESNET_pockets = pd.read_pickle("./results/PURESNET_pockets_DEF_TRANS.pkl")                                    # 2021

DEEPPOCKET_pockets = pd.read_pickle("./results/DeepPocket_pockets_DEF_TRANS2.pkl")                               # 2021

P2RANK_CONS_pockets = pd.read_pickle("./results/P2Rank_CONS_pockets_DEF_TRANS.pkl")                              # 2019
P2RANK_pockets = pd.read_pickle("./results/P2Rank_pockets_DEF_TRANS.pkl")                                        # 2018

FPOCKET_PRANK_CONS_pockets = pd.read_pickle("./results/fpocket_pranked_CONS_pockets_DEF_TRANS.pkl")              # 2009 + 2019
FPOCKET_PRANK_pockets = pd.read_pickle("./results/fpocket_pranked_pockets_DEF_TRANS.pkl")                        # 2009 + 2015
FPOCKET_pockets = pd.read_pickle("./results/fpocket_pockets_DEF_TRANS.pkl")                                      # 2009

POCKETFINDER_PRANK_CONS_pockets = pd.read_pickle("./results/pocketfinder_pranked_CONS_pockets_DEF_TRANS.pkl")    # 2005 + 2019
POCKETFINDER_PRANK_pockets = pd.read_pickle("./results/pocketfinder_pranked_pockets_DEF_TRANS.pkl")              # 2005 + 2015
POCKETFINDER_pockets = pd.read_pickle("./results/POCKETFINDER_pockets_DEF_TRANS_UNRANKED.pkl")                   # 2005

LIGSITE_PRANK_CONS_pockets = pd.read_pickle("./results/ligsite_pranked_CONS_pockets_DEF_TRANS.pkl")              # 1997 + 2019    
LIGSITE_PRANK_pockets = pd.read_pickle("./results/ligsite_pranked_pockets_DEF_TRANS.pkl")                        # 1997 + 2015    
LIGSITE_pockets = pd.read_pickle("./results/LIGSITE_pockets_DEF_TRANS_UNRANKED.pkl")                             # 1997

SURFNET_PRANK_CONS_pockets = pd.read_pickle("./results/surfnet_pranked_CONS_pockets_DEF_TRANS.pkl")              # 1995 + 2019
SURFNET_PRANK_pockets = pd.read_pickle("./results/surfnet_pranked_pockets_DEF_TRANS.pkl")                        # 1995 + 2015
SURFNET_pockets = pd.read_pickle("./results/SURFNET_pockets_DEF_TRANS_UNRANKED.pkl")                             # 1995


In [8]:
def get_sites_per_prot_LIGYSIS(df):
    a = df['rep_chain'].value_counts()
    sites_per_prot_df = pd.DataFrame(list(a.items()), columns=['key', 'sites_per_prot'])
    sites_per_prot_df[['origin', 'rep_chain']] = pd.DataFrame(sites_per_prot_df['key'].tolist(), index=sites_per_prot_df.index)
    sites_per_prot_df.drop('key', axis=1, inplace=True)
    sites_per_prot_df = sites_per_prot_df[['origin', 'rep_chain', 'sites_per_prot']]  # Reorder columns
    return df

In [13]:
len(LIGYSIS_sites['rep_chain'].value_counts().to_dict())

3448

In [9]:
sites_per_prot = get_sites_per_prot_LIGYSIS(LIGYSIS_sites)

ValueError: Columns must be same length as key

In [4]:
LIGYSIS_sites_per_prot

NameError: name 'LIGYSIS_sites_per_prot' is not defined

## Adding RANK = ID, as PUResNet does not score pockets

In [28]:
PURESNET_pockets['score'] = PURESNET_pockets['n_aas']

PURESNET_pockets['RANK'] = PURESNET_pockets.ID

## Re-formatting DeepPocket to consider all the re-scored fpocket candidates

In [29]:
DEEPPOCKET_rescored_pockets = DEEPPOCKET_pockets.copy()

DEEPPOCKET_rescored_pockets.drop(
    columns=[
        'score_fpocket', 'drug_score', 'n_alpha_spheres', 'total_sasa',
        'polar_sasa', 'apolar_sasa', 'volume_fpocket', 'mean_hydro_density',
        'mean_alpha_sphere_r', 'mean_alpha_sphere_solvent_acc',
        'apolar_alpha_sphere_prop', 'hydro_score', 'vol_score',
        'polarity_score', 'charge_score', 'prop_polar_atoms',
        'alpha_sphere_density', 'max_dist', 'flexibility', 'RANK_fpocket',
        'centre_deep1', 'centre_fpocket', 'aas_deep', 'n_aas_deep',
        'centre_deep2', 'centre_fpocket_trans', 'RoG_deep',
        'centre_deep_trans', 'centre_mat', 'distance',
        'barycentre_distance', 'centre_fpocket_mat', 'dist_fpocket_mat',
        'up_aas_deep', 'SASA', 'VOL'
    ],
    inplace=True
)

DEEPPOCKET_rescored_pockets.rename(
    columns = {
        "RANK_deep": "RANK",
        "score_deep": "score",
        "n_aas_fpocket": "n_aas",
        "aas_fpocket": "aas",
        "up_aas_fpocket": "up_aas",  
        "RoG_fpocket": "RoG",
        "barycentre_mat": "centre",
    },
    inplace = True
)

### Re-formatting DeepPocket dataframe to drop those fpocket pockets that were not segmented

In [30]:
DEEPPOCKET_segmented_pockets = DEEPPOCKET_pockets.query('n_aas_deep == n_aas_deep').copy().reset_index(drop = True)

DEEPPOCKET_segmented_pockets.drop(
    columns=[
        'score_fpocket', 'drug_score', 'n_alpha_spheres', 'total_sasa',
        'polar_sasa', 'apolar_sasa', 'volume_fpocket', 'mean_hydro_density',
        'mean_alpha_sphere_r', 'mean_alpha_sphere_solvent_acc',
        'apolar_alpha_sphere_prop', 'hydro_score', 'vol_score',
        'polarity_score', 'charge_score', 'prop_polar_atoms',
        'alpha_sphere_density', 'max_dist', 'flexibility',
        'RANK_fpocket','aas_fpocket', 'n_aas_fpocket', 'centre_fpocket', 'RoG_fpocket',  'up_aas_fpocket',
        'centre_deep1', 'centre_deep2','centre_deep_trans','centre_fpocket_trans', 'distance'
       ],
    inplace=True
)

DEEPPOCKET_segmented_pockets.rename(
    columns = {
        "RANK_deep": "RANK",
        "score_deep": "score",
        "aas_deep": "aas",
        "n_aas_deep": "n_aas",
        "centre_mat": "centre",
        "RoG_deep": "RoG",
        "up_aas_deep": "up_aas",        
    },
    inplace = True
)

DEEPPOCKET_segmented_pockets.n_aas = DEEPPOCKET_segmented_pockets.n_aas.astype(int)

DEEPPOCKET_segmented_pockets['RANK'] = DEEPPOCKET_segmented_pockets.groupby('rep_chain')['score'].rank(ascending=False, method='first').astype(int)

## Re-formatting other dataframes to take into account the correct centroid coordinates

In [31]:
## VN-EGNN ##
VNEGNN_pockets.drop(
    columns=[
        'centre', 'centre_trans',
       ],
    inplace=True
)

VNEGNN_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## IF-SITEPRED ##
IFSP_pockets.drop(
    columns=[
        'centre', 'centre_trans',
       ],
    inplace=True
)

IFSP_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## PURESNET ##
PURESNET_pockets.drop(
    columns=[
        'centre',
       ],
    inplace=True
)

PURESNET_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## P2RANK+CONS ##
P2RANK_CONS_pockets.drop(
    columns=[
       'centre_trans',
       ],
    inplace=True
)

## P2RANK ##
P2RANK_pockets.drop(
    columns=[
        'centre', 'centre_trans',
       ],
    inplace=True
)

P2RANK_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## POCKETFINDER ##
POCKETFINDER_pockets.drop(
    columns=[
        'centre'
       ],
    inplace=True
)

POCKETFINDER_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## LIGSITE ##
LIGSITE_pockets.drop(
    columns=[
        'centre'
       ],
    inplace=True
)

LIGSITE_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## SURFNET ##
SURFNET_pockets.drop(
    columns=[
        'centre'
       ],
    inplace=True
)

SURFNET_pockets.rename(
    columns = {
        "centre_mat": "centre",
    },
    inplace = True
)

## Formatting dataframe

In [32]:
LIGYSIS_sites["origin"] = "LIGYSIS"
VNEGNN_pockets["origin"] = "VN-EGNN"
IFSP_pockets["origin"] = "IF-SitePred"
GRASP_pockets["origin"] = "GrASP"

PURESNET_PRANK_CONS_pockets["origin"] = "PUResNet+PRANK+Cons"
PURESNET_PRANK_pockets["origin"] = "PUResNet+PRANK"
PURESNET_pockets["origin"] = "PUResNet"

DEEPPOCKET_segmented_pockets["origin"] = "DeepPocket-Segmented"
DEEPPOCKET_rescored_pockets["origin"] = "DeepPocket-Rescored"

P2RANK_CONS_pockets["origin"] = "P2Rank+Cons"
P2RANK_pockets["origin"] = "P2Rank"

FPOCKET_PRANK_pockets["origin"] = "fpocket+PRANK"
FPOCKET_PRANK_CONS_pockets["origin"] = "fpocket+PRANK+Cons"
FPOCKET_pockets["origin"] = "fpocket"

POCKETFINDER_PRANK_CONS_pockets["origin"] = "PocketFinder+PRANK+Cons"
POCKETFINDER_PRANK_pockets["origin"] = "PocketFinder+PRANK"
POCKETFINDER_pockets["origin"] = "PocketFinder"

LIGSITE_PRANK_CONS_pockets["origin"] = "Ligsite+PRANK+Cons"
LIGSITE_PRANK_pockets["origin"] = "Ligsite+PRANK"
LIGSITE_pockets["origin"] = "Ligsite"

SURFNET_PRANK_CONS_pockets["origin"] = "Surfnet+PRANK+Cons"
SURFNET_PRANK_pockets["origin"] = "Surfnet+PRANK"
SURFNET_pockets["origin"] = "Surfnet"

In [33]:
LIGYSIS_sites['up_aas'] = LIGYSIS_sites['up_aas'].apply(sorted)
VNEGNN_pockets['up_aas'] = VNEGNN_pockets['up_aas'].apply(sorted)
IFSP_pockets['up_aas'] = IFSP_pockets['up_aas'].apply(sorted)
GRASP_pockets['up_aas'] = GRASP_pockets['up_aas'].apply(sorted)
PURESNET_pockets['up_aas'] = PURESNET_pockets['up_aas'].apply(sorted)
DEEPPOCKET_segmented_pockets['up_aas'] = DEEPPOCKET_segmented_pockets['up_aas'].apply(sorted)
DEEPPOCKET_rescored_pockets['up_aas'] = DEEPPOCKET_rescored_pockets['up_aas'].apply(sorted)
P2RANK_CONS_pockets['up_aas'] = P2RANK_CONS_pockets['up_aas'].apply(sorted)
P2RANK_pockets['up_aas'] = P2RANK_pockets['up_aas'].apply(sorted)
FPOCKET_pockets['up_aas'] = FPOCKET_pockets['up_aas'].apply(sorted)
POCKETFINDER_pockets['up_aas'] = POCKETFINDER_pockets['up_aas'].apply(sorted)
LIGSITE_pockets['up_aas'] = LIGSITE_pockets['up_aas'].apply(sorted)
SURFNET_pockets['up_aas'] = SURFNET_pockets['up_aas'].apply(sorted)

In [34]:
ion_prop = read_from_pickle("./../DATA/LIGYSIS_human_sites_ion_prop_APRIL_2024.pkl")

max_dist_per_seg = read_from_pickle("./results/LIGYSIS_max_dist_per_seg.pkl")

prop_mono_per_site = read_from_pickle("./results/LIGYSIS_prop_mono_per_site.pkl")

In [35]:
LIGYSIS_sites.SEG = LIGYSIS_sites.SEG.astype(int)

In [36]:
LIGYSIS_sites["ion_prop"] = LIGYSIS_sites.set_index(['ACC', 'SEG', 'ID']).index.map(ion_prop)
LIGYSIS_sites["cluster_dist"] = LIGYSIS_sites.set_index('SEG_ID').index.map(max_dist_per_seg)
LIGYSIS_sites["monomer_prop"] = LIGYSIS_sites.set_index('SITE_NAME').index.map(prop_mono_per_site)

In [37]:
LIGYSIS_cols =                 ["origin", "rep_chain", "ID",                           "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas", "ion_prop", "cluster_dist", "monomer_prop"]
VNEGNN_cols =                  ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
IFSP_cols =                    ["origin", "rep_chain", "ID", "RANK",      "n_points",  "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
GRASP_cols =                   ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
PURESNET_cols =                ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"] # added score as n_aas. Nevertheless, RANK = ID.
DEEPPOCKET_segmented_cols =    ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
DEEPPOCKET_rescored_cols =     ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG",                "centre", "up_aas", "aas"] # don't have them as they are the same as fpocket ones.
P2RANK_CONS_cols =             ["origin", "rep_chain", "ID", "RANK",      "prob",      "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
P2RANK_cols =                  ["origin", "rep_chain", "ID", "RANK",      "prob",      "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
FPOCKET_cols =                 ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
POCKETFINDER_cols =            ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
LIGSITE_cols =                 ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]
SURFNET_cols =                 ["origin", "rep_chain", "ID", "RANK",      "score",     "n_aas", "RoG", "SASA", "VOL", "centre", "up_aas", "aas"]

In [38]:
len(LIGYSIS_sites.rep_chain.unique())

3448

In [39]:
LIGYSIS_reps = sorted(list(LIGYSIS_sites.rep_chain.unique()))
#save_to_pickle(LIGYSIS_reps, "./results/LIGYSIS_3448_chains.pkl")

## Merging PUResNet data with PRANK-scored pockets

In [40]:
print(len(PURESNET_PRANK_pockets))

print(len(PURESNET_pockets))

3167
3126


In [41]:
PURESNET_PRANK_pockets_filt = PURESNET_PRANK_pockets.merge(
    PURESNET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
PURESNET_PRANK_pockets_filt.n_aas = PURESNET_PRANK_pockets_filt.n_aas.astype(int)

PURESNET_PRANK_pockets_filt[PURESNET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,PUResNet+PRANK,1a52_A,1,1,32.0635,22,8.81,572.40,506.0,"(1.459, -8.69, -1.716)","[343, 346, 347, 349, 350, 353, 383, 384, 387, ...","[47, 50, 51, 53, 54, 57, 87, 88, 91, 92, 95, 9..."
1,PUResNet+PRANK,1a5h_B,1,1,9.4200,24,8.34,790.91,725.0,"(-0.296, 7.11, -7.752)","[357, 400, 401, 402, 403, 484, 507, 508, 509, ...","[47, 90, 91, 92, 93, 174, 197, 198, 199, 200, ..."
2,PUResNet+PRANK,1a9w_E,1,1,56.5060,21,9.56,857.24,886.0,"(-4.619, 5.824, 1.578)","[32, 39, 42, 43, 46, 64, 67, 68, 71, 72, 86, 8...","[31, 38, 41, 42, 45, 63, 66, 67, 70, 71, 85, 8..."


## Merging PUResNet data with PRANK+Cons-scored pockets

In [43]:
PURESNET_PRANK_CONS_pockets_filt = PURESNET_PRANK_CONS_pockets.merge(
    PURESNET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
PURESNET_PRANK_CONS_pockets_filt.n_aas = PURESNET_PRANK_CONS_pockets_filt.n_aas.astype(int)
#PURESNET_PRANK_CONS_pockets_filt.VOL = PURESNET_PRANK_CONS_pockets_filt.VOL.astype(int)

PURESNET_PRANK_CONS_pockets_filt[PURESNET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,PUResNet+PRANK+Cons,1a52_A,1,1,85.9202,22,8.81,572.40,506.0,"(1.459, -8.69, -1.716)","[343, 346, 347, 349, 350, 353, 383, 384, 387, ...","[47, 50, 51, 53, 54, 57, 87, 88, 91, 92, 95, 9..."
1,PUResNet+PRANK+Cons,1a5h_B,1,1,20.5193,24,8.34,790.91,725.0,"(-0.296, 7.11, -7.752)","[357, 400, 401, 402, 403, 484, 507, 508, 509, ...","[47, 90, 91, 92, 93, 174, 197, 198, 199, 200, ..."
2,PUResNet+PRANK+Cons,1a9w_E,1,1,110.2308,21,9.56,857.24,886.0,"(-4.619, 5.824, 1.578)","[32, 39, 42, 43, 46, 64, 67, 68, 71, 72, 86, 8...","[31, 38, 41, 42, 45, 63, 66, 67, 70, 71, 85, 8..."


## Merging fpocket data with PRANK-rescored fpocket candidates

In [49]:
FPOCKET_PRANK_pockets_filt = FPOCKET_PRANK_pockets.merge(
    FPOCKET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
FPOCKET_PRANK_pockets_filt.n_aas = FPOCKET_PRANK_pockets_filt.n_aas.astype(int)
#FPOCKET_PRANK_pockets_filt.VOL = FPOCKET_PRANK_pockets_filt.VOL.astype(int)

FPOCKET_PRANK_pockets_filt[FPOCKET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,fpocket+PRANK,1a52_A,1,1,44.2707,32,11.82,1388.33,1981.0,"(-4.199, -6.401, 0.526)","[343, 346, 347, 349, 350, 351, 353, 354, 358, ...","[47, 50, 51, 53, 54, 55, 57, 58, 62, 66, 71, 7..."
1,fpocket+PRANK,1a52_A,11,2,5.9334,8,6.03,428.48,341.0,"(-3.074, -18.658, -1.952)","[340, 343, 344, 347, 525, 528, 529, 532]","[44, 47, 48, 51, 229, 232, 233, 236]"
2,fpocket+PRANK,1a52_A,4,3,3.6748,15,7.62,559.21,399.0,"(1.479, -0.948, 6.948)","[323, 324, 325, 326, 327, 353, 356, 357, 386, ...","[27, 28, 29, 30, 31, 57, 60, 61, 90, 91, 94, 9..."


## Merging fpocket data with PRANK+Cons-rescored fpocket candidates

In [50]:
FPOCKET_PRANK_CONS_pockets_filt = FPOCKET_PRANK_CONS_pockets.merge(
    FPOCKET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
FPOCKET_PRANK_CONS_pockets_filt.n_aas = FPOCKET_PRANK_CONS_pockets_filt.n_aas.astype(int)
#FPOCKET_PRANK_CONS_pockets_filt.VOL = FPOCKET_PRANK_CONS_pockets_filt.VOL.astype(int)

FPOCKET_PRANK_CONS_pockets_filt[FPOCKET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,fpocket+PRANK+Cons,1a52_A,1,1,76.0657,32,11.82,1388.33,1981.0,"(-4.199, -6.401, 0.526)","[343, 346, 347, 349, 350, 351, 353, 354, 358, ...","[47, 50, 51, 53, 54, 55, 57, 58, 62, 66, 71, 7..."
1,fpocket+PRANK+Cons,1a52_A,11,2,13.4611,8,6.03,428.48,341.0,"(-3.074, -18.658, -1.952)","[340, 343, 344, 347, 525, 528, 529, 532]","[44, 47, 48, 51, 229, 232, 233, 236]"
2,fpocket+PRANK+Cons,1a52_A,6,3,3.0971,14,7.75,1063.41,240.0,"(-19.512, 6.829, -3.23)","[371, 373, 374, 376, 377, 380, 460, 461, 462, ...","[75, 77, 78, 80, 81, 84, 164, 165, 166, 168, 1..."


## Merging PocketFinder data with PRANK-scored pockets

In [55]:
POCKETFINDER_PRANK_pockets_filt = POCKETFINDER_PRANK_pockets.merge(
    POCKETFINDER_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
POCKETFINDER_PRANK_pockets_filt.n_aas = POCKETFINDER_PRANK_pockets_filt.n_aas.astype(int)
#POCKETFINDER_PRANK_pockets_filt.VOL = POCKETFINDER_PRANK_pockets_filt.VOL.astype(int)

POCKETFINDER_PRANK_pockets_filt[POCKETFINDER_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,PocketFinder+PRANK,1a52_A,3,1,30.4977,12,7.74,763.94,268.0,"(14.979, -19.541, 4.818)","[331, 333, 335, 336, 337, 407, 408, 409, 410, ...","[35, 37, 39, 40, 41, 111, 112, 113, 114, 115, ..."
1,PocketFinder+PRANK,1a52_A,2,2,7.2364,14,6.89,986.00,174.0,"(-22.231, 6.663, -4.167)","[371, 373, 374, 377, 460, 461, 462, 463, 464, ...","[75, 77, 78, 81, 164, 165, 166, 167, 168, 169,..."
2,PocketFinder+PRANK,1a52_A,4,3,5.8188,30,9.53,1141.78,1136.0,"(8.006, 1.628, 7.98)","[320, 323, 324, 325, 326, 327, 353, 354, 356, ...","[24, 27, 28, 29, 30, 31, 57, 58, 60, 61, 64, 9..."


## Merging PocketFinder data with PRANK+Cons-rescored pockets

In [44]:
POCKETFINDER_PRANK_CONS_pockets_filt = POCKETFINDER_PRANK_CONS_pockets.merge(
    POCKETFINDER_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
POCKETFINDER_PRANK_CONS_pockets_filt.n_aas = POCKETFINDER_PRANK_CONS_pockets_filt.n_aas.astype(int)
#POCKETFINDER_PRANK_CONS_pockets_filt.VOL = POCKETFINDER_PRANK_CONS_pockets_filt.VOL.astype(int)

POCKETFINDER_PRANK_CONS_pockets_filt[POCKETFINDER_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,PocketFinder+PRANK+Cons,1a52_A,3,1,77.6260,12,7.74,763.94,268.0,"(14.979, -19.541, 4.818)","[331, 333, 335, 336, 337, 407, 408, 409, 410, ...","[35, 37, 39, 40, 41, 111, 112, 113, 114, 115, ..."
1,PocketFinder+PRANK+Cons,1a52_A,4,2,9.2178,30,9.53,1141.78,1136.0,"(8.006, 1.628, 7.98)","[320, 323, 324, 325, 326, 327, 353, 354, 356, ...","[24, 27, 28, 29, 30, 31, 57, 58, 60, 61, 64, 9..."
2,PocketFinder+PRANK+Cons,1a52_A,2,3,8.4033,14,6.89,986.00,174.0,"(-22.231, 6.663, -4.167)","[371, 373, 374, 377, 460, 461, 462, 463, 464, ...","[75, 77, 78, 81, 164, 165, 166, 167, 168, 169,..."


## Merging Ligsite data with PRANK-scored pockets

In [58]:
LIGSITE_PRANK_pockets_filt = LIGSITE_PRANK_pockets.merge(
    LIGSITE_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
LIGSITE_PRANK_pockets_filt.n_aas = LIGSITE_PRANK_pockets_filt.n_aas.astype(int)
#LIGSITE_PRANK_pockets_filt.VOL = LIGSITE_PRANK_pockets_filt.VOL.astype(int)

LIGSITE_PRANK_pockets_filt[LIGSITE_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,Ligsite+PRANK,1a52_A,1,1,35.8552,46,10.07,1369.28,919.0,"(-0.015, -9.913, -1.197)","[342, 343, 344, 345, 346, 347, 348, 349, 350, ...","[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5..."
1,Ligsite+PRANK,1a52_A,2,2,7.4068,37,9.12,1309.35,1033.0,"(2.33, -1.69, 7.264)","[323, 324, 325, 326, 327, 328, 329, 349, 350, ...","[27, 28, 29, 30, 31, 32, 33, 53, 54, 56, 57, 5..."
2,Ligsite+PRANK,1a5h_B,2,1,14.4832,51,10.19,1897.66,1412.0,"(2.741, 13.688, -4.192)","[312, 357, 403, 441, 446, 454, 468, 471, 472, ...","[2, 47, 93, 131, 136, 144, 158, 161, 162, 164,..."


## Merging Ligsite data with PRANK+Cons-scored pockets

In [45]:
LIGSITE_PRANK_CONS_pockets_filt = LIGSITE_PRANK_CONS_pockets.merge(
    LIGSITE_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
LIGSITE_PRANK_CONS_pockets_filt.n_aas = LIGSITE_PRANK_CONS_pockets_filt.n_aas.astype(int)
#LIGSITE_PRANK_CONS_pockets_filt.VOL = LIGSITE_PRANK_CONS_pockets_filt.VOL.astype(int)

LIGSITE_PRANK_CONS_pockets_filt[LIGSITE_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,Ligsite+PRANK+Cons,1a52_A,1,1,85.4594,46,10.07,1369.28,919.0,"(-0.015, -9.913, -1.197)","[342, 343, 344, 345, 346, 347, 348, 349, 350, ...","[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5..."
1,Ligsite+PRANK+Cons,1a52_A,2,2,13.8353,37,9.12,1309.35,1033.0,"(2.33, -1.69, 7.264)","[323, 324, 325, 326, 327, 328, 329, 349, 350, ...","[27, 28, 29, 30, 31, 32, 33, 53, 54, 56, 57, 5..."
2,Ligsite+PRANK+Cons,1a5h_B,2,1,26.5277,51,10.19,1897.66,1412.0,"(2.741, 13.688, -4.192)","[312, 357, 403, 441, 446, 454, 468, 471, 472, ...","[2, 47, 93, 131, 136, 144, 158, 161, 162, 164,..."


## Merging Surfnet data with PRANK-rescored pockets

In [62]:
SURFNET_PRANK_pockets_filt = SURFNET_PRANK_pockets.merge(
    SURFNET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
SURFNET_PRANK_pockets_filt.n_aas = SURFNET_PRANK_pockets_filt.n_aas.astype(int)
#SURFNET_PRANK_pockets_filt.VOL = SURFNET_PRANK_pockets_filt.VOL.astype(int)

SURFNET_PRANK_pockets_filt[SURFNET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,Surfnet+PRANK,1a52_A,3,1,35.9118,16,6.96,1164.77,174.0,"(-22.143, 7.88, -3.968)","[371, 373, 374, 377, 460, 461, 462, 463, 464, ...","[75, 77, 78, 81, 164, 165, 166, 167, 168, 169,..."
1,Surfnet+PRANK,1a52_A,2,2,7.6292,37,9.71,1167.12,827.0,"(0.607, -10.043, -1.592)","[342, 343, 345, 346, 347, 348, 349, 350, 351, ...","[46, 47, 49, 50, 51, 52, 53, 54, 55, 57, 87, 8..."
2,Surfnet+PRANK,1a52_A,1,3,5.1668,16,7.54,675.43,531.0,"(-2.263, 8.398, -9.67)","[385, 451, 452, 455, 456, 459, 479, 507, 508, ...","[89, 155, 156, 159, 160, 163, 183, 211, 212, 2..."


## Merging Surfnet data with PRANK+Cons-scored pockets

In [61]:
SURFNET_PRANK_CONS_pockets_filt = SURFNET_PRANK_CONS_pockets.merge(
    SURFNET_pockets[['rep_chain', 'ID', 'n_aas', 'RoG', 'SASA', 'VOL', 'centre', 'up_aas', 'aas']],
    on=['rep_chain', 'ID'],
    how='left'
).query('n_aas == n_aas').reset_index(drop=True) #.dropna()
SURFNET_PRANK_CONS_pockets_filt.n_aas = SURFNET_PRANK_CONS_pockets_filt.n_aas.astype(int)
#SURFNET_PRANK_CONS_pockets_filt.VOL = SURFNET_PRANK_CONS_pockets_filt.VOL.astype(int)

SURFNET_PRANK_CONS_pockets_filt[SURFNET_cols].head(3)

,origin,rep_chain,ID,RANK,score,n_aas,RoG,SASA,VOL,centre,up_aas,aas
0,Surfnet+PRANK+Cons,1a52_A,3,1,97.3518,16,6.96,1164.77,174.0,"(-22.143, 7.88, -3.968)","[371, 373, 374, 377, 460, 461, 462, 463, 464, ...","[75, 77, 78, 81, 164, 165, 166, 167, 168, 169,..."
1,Surfnet+PRANK+Cons,1a52_A,1,2,11.2193,16,7.54,675.43,531.0,"(-2.263, 8.398, -9.67)","[385, 451, 452, 455, 456, 459, 479, 507, 508, ...","[89, 155, 156, 159, 160, 163, 183, 211, 212, 2..."
2,Surfnet+PRANK+Cons,1a52_A,2,3,8.2490,37,9.71,1167.12,827.0,"(0.607, -10.043, -1.592)","[342, 343, 345, 346, 347, 348, 349, 350, 351, ...","[46, 47, 49, 50, 51, 52, 53, 54, 55, 57, 87, 8..."


### Filter out structures for which SIFTS mapping to UniProt residues in predicted sites fail

In [47]:
bad_reps = read_from_pickle("./results/OLD/other_errrors_reps.pkl")

In [63]:
no_sifts_vnegnn = read_from_pickle("./results/VNEGNN_no_sifts_TRANS.pkl")
no_sifts_ifsp = read_from_pickle("./results/IFSP_no_sifts_TRANS.pkl")
no_sifts_grasp = read_from_pickle("./results/GrASP_no_sifts_TRANS.pkl")
no_sifts_puresnet = read_from_pickle("./results/PURESNET_no_sifts.pkl")
no_sifts_deeppocket = read_from_pickle("./results/DeepPocket_no_sifts_TRANS.pkl")
no_sifts_p2rank_cons = read_from_pickle("./results/P2Rank_CONS_no_sifts_TRANS.pkl")
no_sifts_p2rank = read_from_pickle("./results/P2Rank_no_sifts_TRANS.pkl")
no_sifts_fpocket = read_from_pickle("./results/fpocket_no_sifts_TRANS.pkl")
no_sifts_pocketfinder = read_from_pickle("./results/POCKETFINDER_no_sifts.pkl")
no_sifts_ligsite = read_from_pickle("./results/LIGSITE_no_sifts.pkl")
no_sifts_surfnet = read_from_pickle("./results/SURFNET_no_sifts.pkl")

no_sifts_un = sorted(
    list(
        set(
            no_sifts_vnegnn + no_sifts_ifsp + no_sifts_grasp + no_sifts_puresnet + no_sifts_deeppocket + no_sifts_p2rank_cons + no_sifts_p2rank + no_sifts_fpocket + no_sifts_pocketfinder + no_sifts_ligsite + no_sifts_surfnet 
        )
    )
)
print(len(no_sifts_un))

LIGYSIS_sites_filt = LIGYSIS_sites.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
VNEGNN_pockets_filt = VNEGNN_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
IFSP_pockets_filt = IFSP_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
GRASP_pockets_filt = GRASP_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

PURESNET_PRANK_CONS_pockets_filt = PURESNET_PRANK_CONS_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
PURESNET_PRANK_pockets_filt = PURESNET_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
PURESNET_pockets_filt = PURESNET_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

DEEPPOCKET_segmented_pockets_filt = DEEPPOCKET_segmented_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
DEEPPOCKET_rescored_pockets_filt = DEEPPOCKET_rescored_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

P2RANK_CONS_pockets_filt = P2RANK_CONS_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
P2RANK_pockets_filt = P2RANK_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

FPOCKET_PRANK_CONS_pockets_filt = FPOCKET_PRANK_CONS_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
FPOCKET_PRANK_pockets_filt = FPOCKET_PRANK_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
FPOCKET_pockets_filt = FPOCKET_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

POCKETFINDER_PRANK_CONS_pockets_filt = POCKETFINDER_PRANK_CONS_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
POCKETFINDER_PRANK_pockets_filt = POCKETFINDER_PRANK_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
POCKETFINDER_pockets_filt = POCKETFINDER_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

LIGSITE_PRANK_CONS_pockets_filt = LIGSITE_PRANK_CONS_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
LIGSITE_PRANK_pockets_filt = LIGSITE_PRANK_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
LIGSITE_pockets = LIGSITE_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

SURFNET_PRANK_CONS_pockets_filt = SURFNET_PRANK_CONS_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
SURFNET_PRANK_pockets_filt = SURFNET_PRANK_pockets_filt.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)
SURFNET_pockets_filt = SURFNET_pockets.query('rep_chain not in @no_sifts_un').copy().reset_index(drop = True)

990


In [24]:
save_to_pickle(no_sifts_un, "./results/ALL_no_sifts_TRANS.pkl")

In [64]:
LIGYSIS_sites_filt = LIGYSIS_sites_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
VNEGNN_pockets_filt = VNEGNN_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
IFSP_pockets_filt = IFSP_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
GRASP_pockets_filt = GRASP_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

PURESNET_PRANK_CONS_pockets_filt = PURESNET_PRANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
PURESNET_PRANK_pockets_filt = PURESNET_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
PURESNET_pockets_filt = PURESNET_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

DEEPPOCKET_segmented_pockets_filt = DEEPPOCKET_segmented_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
DEEPPOCKET_rescored_pockets_filt = DEEPPOCKET_rescored_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

P2RANK_CONS_pockets_filt = P2RANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
P2RANK_pockets_filt = P2RANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

FPOCKET_PRANK_CONS_pockets_filt = FPOCKET_PRANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
FPOCKET_PRANK_pockets_filt = FPOCKET_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
FPOCKET_pockets_filt = FPOCKET_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

POCKETFINDER_PRANK_CONS_pockets_filt = POCKETFINDER_PRANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
POCKETFINDER_PRANK_pockets_filt = POCKETFINDER_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
POCKETFINDER_pockets_filt = POCKETFINDER_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

LIGSITE_PRANK_CONS_pockets_filt = LIGSITE_PRANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
LIGSITE_PRANK_pockets_filt = LIGSITE_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
LIGSITE_pockets = LIGSITE_pockets.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

SURFNET_PRANK_CONS_pockets_filt = SURFNET_PRANK_CONS_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
SURFNET_PRANK_pockets_filt = SURFNET_PRANK_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)
SURFNET_pockets_filt = SURFNET_pockets_filt.query('rep_chain not in @bad_reps').copy().reset_index(drop = True)

In [65]:
len(LIGYSIS_sites_filt.ACC.unique())

2565

In [66]:
LIGYSIS_site_reps = LIGYSIS_sites_filt.rep_chain.unique().tolist()
print(len(LIGYSIS_site_reps)) #2775

2775


In [70]:
VNEGNN_pockets_filt = VNEGNN_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
IFSP_pockets_filt = IFSP_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
GRASP_pockets_filt = GRASP_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

PURESNET_PRANK_CONS_pockets_filt = PURESNET_PRANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
PURESNET_PRANK_pockets_filt = PURESNET_PRANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
PURESNET_pockets_filt = PURESNET_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

DEEPPOCKET_segmented_pockets_filt = DEEPPOCKET_segmented_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)                                                                                                                            
DEEPPOCKET_rescored_pockets_filt = DEEPPOCKET_rescored_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

P2RANK_CONS_pockets_filt = P2RANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
P2RANK_pockets_filt = P2RANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

FPOCKET_PRANK_CONS_pockets_filt = FPOCKET_PRANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
FPOCKET_PRANK_pockets_filt = FPOCKET_PRANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
FPOCKET_pockets_filt = FPOCKET_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

POCKETFINDER_PRANK_CONS_pockets_filt = POCKETFINDER_PRANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
POCKETFINDER_PRANK_pockets_filt = POCKETFINDER_PRANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
POCKETFINDER_pockets_filt = POCKETFINDER_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

LIGSITE_PRANK_CONS_pockets_filt = LIGSITE_PRANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
LIGSITE_PRANK_pockets_filt = LIGSITE_PRANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
LIGSITE_pockets = LIGSITE_pockets.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

SURFNET_PRANK_CONS_pockets_filt = SURFNET_PRANK_CONS_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
SURFNET_PRANK_pockets_filt = SURFNET_PRANK_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)
SURFNET_pockets_filt = SURFNET_pockets_filt.query('rep_chain in @LIGYSIS_site_reps').copy().reset_index(drop = True)

In [72]:
combined_df = pd.concat(
    [
        LIGYSIS_sites_filt[LIGYSIS_cols],
        VNEGNN_pockets_filt[VNEGNN_cols],
        IFSP_pockets_filt[IFSP_cols],
        GRASP_pockets_filt[GRASP_cols],
        
        PURESNET_PRANK_CONS_pockets_filt[PURESNET_cols], 
        PURESNET_PRANK_pockets_filt[PURESNET_cols],
        PURESNET_pockets_filt[PURESNET_cols],
        
        DEEPPOCKET_segmented_pockets_filt[DEEPPOCKET_segmented_cols],
        DEEPPOCKET_rescored_pockets_filt[DEEPPOCKET_rescored_cols],
        
        P2RANK_CONS_pockets_filt[P2RANK_CONS_cols],
        P2RANK_pockets_filt[P2RANK_cols],
        
        FPOCKET_PRANK_CONS_pockets_filt[FPOCKET_cols], 
        FPOCKET_PRANK_pockets_filt[FPOCKET_cols], 
        FPOCKET_pockets_filt[FPOCKET_cols],
        
        POCKETFINDER_PRANK_CONS_pockets_filt[POCKETFINDER_cols], 
        POCKETFINDER_PRANK_pockets_filt[POCKETFINDER_cols], 
        POCKETFINDER_pockets_filt[POCKETFINDER_cols],  
        
        LIGSITE_PRANK_CONS_pockets_filt[LIGSITE_cols], 
        LIGSITE_PRANK_pockets_filt[LIGSITE_cols], 
        LIGSITE_pockets[LIGSITE_cols],  
        
        SURFNET_PRANK_CONS_pockets_filt[SURFNET_cols], 
        SURFNET_PRANK_pockets_filt[SURFNET_cols], 
        SURFNET_pockets_filt[SURFNET_cols],  
    ]
).reset_index(drop = True)

In [74]:
combined_df_filt = combined_df.query('n_aas > 0').copy().reset_index(drop = True)

In [75]:
len(combined_df_filt.rep_chain.unique().tolist())

2775

In [77]:
combined_df_filt.query('VOL != VOL').origin.value_counts()

DeepPocket-Rescored    57815
LIGYSIS                    2
Ligsite+PRANK+Cons         1
Ligsite+PRANK              1
Ligsite                    1
Name: origin, dtype: int64

In [78]:
combined_df_filt.to_pickle("./results/ALL_PREDS_COMBINED_RIGHT_EXTENDED_DEF.pkl")